# Mobility-Station-Finder

## Imports

In [1]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from matrixconverters.read_ptv import ReadPTVMatrix
import requests
import os

## Read and process static data

### Paths

In [2]:
path_npvm_zones = os.path.join("..", "data", "Verkehrszonen_Schweiz_NPVM_2017_shp.zip")
path_mobility_stations = os.path.join("..", "data", "mobility-stationen-und-fahrzeuge-schweiz.csv")
path_pt_jrta = os.path.join("..", "data", "140_JRTA_(OEV).mtx")
path_pt_ntr = os.path.join("..", "data", "144_NTR_(OEV).mtx")

### Read NPVM-zones with shapes

In [3]:
%time gdf_npvm_zones = gpd.read_file(path_npvm_zones, encoding="cp1252").to_crs(4326)

CPU times: total: 3.44 s
Wall time: 3.44 s


In [4]:
gdf_npvm_zones.head()

,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry
0,101001,1,1,Aeugst am Albis,1,None,1,ZH,3,Ländlich,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.47334 47.26128, 8.47334 47.26139, ..."
1,201001,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.42224 47.29775, 8.42282 47.29816, ..."
2,201002,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.44770 47.26794, 8.44767 47.26782, ..."
3,201003,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.43834 47.27714, 8.43814 47.27726, ..."
4,201004,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.45000 47.27949, 8.45007 47.27945, ..."


In [5]:
print("Anzahl NPVM-Zonen: {}".format(len(gdf_npvm_zones)))

Anzahl NPVM-Zonen: 7978


In [6]:
def get_npvm_zone(id_):
    return gdf_npvm_zones[gdf_npvm_zones.ID == id_]

In [7]:
get_npvm_zone(35101026)

,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry
1324,35101026,35103,351,Bern,1,None,2,BE,1,Städtisch,351,Bern,6012,Bern,"POLYGON ((7.42354 46.93420, 7.42336 46.93393, ..."


### Read Mobility-stations, assign NPVM-zones to Mobility-stations

In [14]:
df_mobility_vechicles = pd.read_csv(path_mobility_stations, delimiter=";", encoding="utf8")[["Stationsnummer", "Name", "Standort"]].dropna()

In [15]:
print("Anzahl Mobility Fahrzeuge: {}".format(len(df_mobility_vechicles)))

Anzahl Mobility Fahrzeuge: 2662


In [16]:
df_mobility_stations = df_mobility_vechicles.groupby("Stationsnummer").first().reset_index()

In [17]:
df_mobility_stations["lon"] = df_mobility_stations["Standort"].apply(lambda x: x.split(",")[1])
df_mobility_stations["lat"] = df_mobility_stations["Standort"].apply(lambda x: x.split(",")[0])
df_mobility_stations = gpd.GeoDataFrame(df_mobility_stations, geometry=gpd.points_from_xy(df_mobility_stations.lon, df_mobility_stations.lat), crs=4326)

In [18]:
print("Anzahl Mobility Stationen: {}".format(len(df_mobility_stations)))

Anzahl Mobility Stationen: 1550


In [19]:
gdf_mobilty_stations_with_npvm_zone = gpd.sjoin(df_mobility_stations, gdf_npvm_zones)[["Stationsnummer", "Name", "geometry", "ID", "N_Gem"]]

In [20]:
print("Anzahl Mobility Standorte mit zugeordneter NPVM-Zone: {}".format(len(gdf_mobilty_stations_with_npvm_zone)))

Anzahl Mobility Standorte mit zugeordneter NPVM-Zone: 1549


In [21]:
gdf_mobilty_stations_with_npvm_zone.head()

,Stationsnummer,Name,geometry,ID,N_Gem
0,1006,Brugg Bahnhof,POINT (8.20942 47.48154),409501008,Brugg
719,3215,Brugg Post Neumarkt / Bahnhofstrasse,POINT (8.20757 47.48216),409501008,Brugg
1,1012,Arbon Bahnhof,POINT (9.43345 47.51032),440101010,Arbon
2,1019,Basel Vogesenstrasse,POINT (7.57483 47.56869),270101028,Basel
3,1024,Bellinzona Stazione,POINT (9.03017 46.19630),500201015,Bellinzona


In [22]:
gdf_npvm_zones_with_mobility_station = gdf_mobilty_stations_with_npvm_zone.dissolve(by="ID", aggfunc={"N_Gem": "first", "Name": lambda x: list(x), "Stationsnummer": lambda x: list(x)}).reset_index()

In [23]:
print("Anzahl NPVM-Zonen mit Mobility-Standort: {}".format(len(gdf_npvm_zones_with_mobility_station)))

Anzahl NPVM-Zonen mit Mobility-Standort: 1311


### Read PT-skims

In [24]:
%time skim_jrta = ReadPTVMatrix(path_pt_jrta)

CPU times: total: 6.91 s
Wall time: 6.94 s


In [40]:
len(skim_jrta)

3

In [25]:
%time skim_ntr = ReadPTVMatrix(path_pt_ntr)

CPU times: total: 7.73 s
Wall time: 7.73 s


In [26]:
def get_skim(from_npvm_zone_id, to_npvm_zone_id, skim_matrix):
    return skim_matrix.sel(origins=from_npvm_zone_id).sel(destinations=to_npvm_zone_id).matrix.item()

In [27]:
def get_jrta(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(from_npvm_zone_id, to_npvm_zone_id, skim_jrta)

In [28]:
def get_ntr(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(from_npvm_zone_id, to_npvm_zone_id, skim_ntr)

## Execute query

### Constants

In [41]:
GEOMETRY = "geometry"
EASTING = "easting"
NORTHING = "northing"

NPVM_ID = "ID"
STATIONSNUMMER = "Stationsnummer"
MIV_DISTANZ_BIS_ZIEL_KM = "MIV_Distanz_bis_Ziel_km"
MIV_ZEIT_BIS_ZIEL_MIN = "MIV_Zeit_bis_Ziel_min"

DISTANCES = "distances"
DURATIONS = "durations"

OEV_JRTA_VON_START_MIN = "OEV_JRTA_von_Start_min"
OEV_NTR_VON_START = "OEV_NTR_von_Start"

KOSTEN_CHF = "Kosten_CHF"

CHF_PER_KM_MOBILITY = 0.75
MIN_PER_TRANSFER = 20.0
FILTER_FACTOR = 1.05

OUTPUT_TYPE_GDF = "gdf"
OUTPUT_TYPE_DICT = "dict"

### Functions

In [42]:
def get_gdf_point_with_npvm_zone_id(point_easting_northing, gdf_npvm_zones):
    point = Point(point_easting_northing[0], point_easting_northing[1])
    gdf_point = gpd.GeoDataFrame({'geometry': [point]}, crs="EPSG:4326")
    gdf_point_with_zone = gpd.sjoin(gdf_point, gdf_npvm_zones)[["ID", "N_Gem", "geometry"]]
    return gdf_point_with_zone

In [43]:
def get_npvm_zone_id(gdf_point_with_npvm_zone_id):
    if len(gdf_point_with_npvm_zone_id) != 1:
        raise ValueError("only one entry exprected, but there are {}".format(len(gdf_point_with_npvm_zone_id)))
    return gdf_point_with_npvm_zone_id["ID"].item()

In [44]:
def get_potential_mobility_stations(gdf_orig_with_npvm_zone_id, gdf_dest_with_npvm_zone_id, gdf_mobilty_stations_with_npvm_zone, factor=1.5, constant=30.0):
    orig_zone_id = get_npvm_zone_id(gdf_orig_with_npvm_zone_id)
    dest_zone_id = get_npvm_zone_id(gdf_dest_with_npvm_zone_id)
    jrta_orig_dest = get_jrta(orig_zone_id, dest_zone_id)
    potential_stations_ids = []
    for station_id, zone_id in gdf_mobilty_stations_with_npvm_zone[["Stationsnummer", "ID"]].values.tolist():
        jrta_orig_station = get_jrta(orig_zone_id, zone_id)
        jrta_station_dest = get_jrta(zone_id, dest_zone_id)
        if jrta_orig_station + jrta_station_dest <= factor * jrta_orig_dest + constant:
            potential_stations_ids += [station_id]
    df_potential_station_ids = pd.DataFrame(potential_stations_ids, columns=["Stationsnummer"])
    return pd.merge(gdf_mobilty_stations_with_npvm_zone, df_potential_station_ids, on=["Stationsnummer"])

In [45]:
def collect_data_on_potential_npvm_zones(gdf_orig_with_npvm_zone_id, gdf_dest_with_npvm_zone_id, gdf_potential_mobility_stations):
    list_potential_mobility_stations = list(gdf_potential_mobility_stations.to_dict("records"))
    coords_str = "{},{}".format(gdf_dest_with_npvm_zone_id[GEOMETRY].x.item(), gdf_dest_with_npvm_zone_id[GEOMETRY].y.item())
    for pot_mob_st in list_potential_mobility_stations:
        center = pot_mob_st[GEOMETRY].centroid
        coords_str += ";{},{}".format(center.x, center.y)
    url = "https://router.project-osrm.org/table/v1/driving/{}?destinations=0&annotations=duration,distance".format(coords_str)
    res = requests.get(url).json()
    road_distances_from_potential_mobility_station_to_dest_per_stationsnummer = {x[STATIONSNUMMER]: res[DISTANCES][n + 1][0] for n, x in enumerate(list_potential_mobility_stations)}
    road_durations_from_potential_mobility_station_to_dest_per_stationsnummer = {x[STATIONSNUMMER]: res[DURATIONS][n + 1][0] for n, x in enumerate(list_potential_mobility_stations)}
    
    pd_distances = pd.DataFrame(list(road_distances_from_potential_mobility_station_to_dest_per_stationsnummer.items()), columns=[STATIONSNUMMER, MIV_DISTANZ_BIS_ZIEL_KM])
    pd_distances[MIV_DISTANZ_BIS_ZIEL_KM] = pd_distances[MIV_DISTANZ_BIS_ZIEL_KM] / 1000.0
    
    pd_durations= pd.DataFrame(list(road_durations_from_potential_mobility_station_to_dest_per_stationsnummer.items()), columns=[STATIONSNUMMER, MIV_ZEIT_BIS_ZIEL_MIN])
    pd_durations[MIV_ZEIT_BIS_ZIEL_MIN] = pd_durations[MIV_ZEIT_BIS_ZIEL_MIN] / 60.0

    gdf_potential_mobility_stations_with_data = pd.merge(gdf_potential_mobility_stations, pd_distances, on=[STATIONSNUMMER])
    gdf_potential_mobility_stations_with_data = pd.merge(gdf_potential_mobility_stations_with_data, pd_durations, on=[STATIONSNUMMER])
    
    
    zone_ids_list = set(x.item() for x in gdf_potential_mobility_stations[[NPVM_ID]].values)
    orig_zone_id = get_npvm_zone_id(gdf_orig_with_npvm_zone_id)
    jrta_list = [(x, get_jrta(orig_zone_id, x)) for x in zone_ids_list]
    ntr_list = [(x, get_ntr(orig_zone_id, x)) for x in zone_ids_list]
    
    pd_jrtas = pd.DataFrame(jrta_list, columns=[NPVM_ID, OEV_JRTA_VON_START_MIN])
    pd_ntrs = pd.DataFrame(ntr_list, columns=[NPVM_ID, OEV_NTR_VON_START])
    
    gdf_potential_mobility_stations_with_data = pd.merge(gdf_potential_mobility_stations_with_data, pd_jrtas, on=[NPVM_ID])
    gdf_potential_mobility_stations_with_data = pd.merge(gdf_potential_mobility_stations_with_data, pd_ntrs, on=[NPVM_ID])
    if len(gdf_potential_mobility_stations) != len(gdf_potential_mobility_stations_with_data):
        raise ValueError("# mobility stations has changed")
    return gdf_potential_mobility_stations_with_data

In [46]:
def calc_generalized_costs(gdf_potential_mobility_stations_with_data, vtt_chf_per_h=20):
    gdf_potential_mobility_stations_with_data[KOSTEN_CHF] = \
        CHF_PER_KM_MOBILITY * gdf_potential_mobility_stations_with_data[MIV_DISTANZ_BIS_ZIEL_KM] + \
        (gdf_potential_mobility_stations_with_data[MIV_ZEIT_BIS_ZIEL_MIN] + gdf_potential_mobility_stations_with_data[OEV_JRTA_VON_START_MIN] + 
        gdf_potential_mobility_stations_with_data[OEV_NTR_VON_START] * MIN_PER_TRANSFER) / 60.0 * vtt_chf_per_h
    return gdf_potential_mobility_stations_with_data.sort_values(by=KOSTEN_CHF, ascending=True)

In [47]:
def calc_best_mobility_stations_per_vtt(gdf_potential_mobility_stations_with_data, vtt_chf_per_h, output_type=OUTPUT_TYPE_GDF):
    df_tmp = calc_generalized_costs(gdf_potential_mobility_stations_with_data, vtt_chf_per_h=vtt_chf_per_h)
    min_cost = df_tmp[KOSTEN_CHF].min()
    df_tmp = df_tmp[df_tmp[KOSTEN_CHF] <= min_cost * FILTER_FACTOR]
    df_tmp = pd.merge(gdf_potential_mobility_stations_with_data, df_tmp[[NPVM_ID]], on=NPVM_ID).sort_values(by=KOSTEN_CHF)
    if output_type == OUTPUT_TYPE_GDF:
        return df_tmp
    elif output_type == OUTPUT_TYPE_DICT:
        return df_tmp.to_dict("records")

In [48]:
def get_best_mobility_statons_per_vtt(orig_lon_lat, dest_lon_lat, output_type=OUTPUT_TYPE_GDF):
    gdf_orig_with_npvm_zone_id = get_gdf_point_with_npvm_zone_id(orig_lon_lat, gdf_npvm_zones)
    gdf_dest_with_npvm_zone_id = get_gdf_point_with_npvm_zone_id(dest_lon_lat, gdf_npvm_zones)
    gdf_potential_mobility_stations = get_potential_mobility_stations(gdf_orig_with_npvm_zone_id, gdf_dest_with_npvm_zone_id, gdf_mobilty_stations_with_npvm_zone)
    gdf_potential_mobility_stations_with_data = collect_data_on_potential_npvm_zones(gdf_orig_with_npvm_zone_id, gdf_dest_with_npvm_zone_id, gdf_potential_mobility_stations)
    best_mobility_stations_per_vtt = {vtt: calc_best_mobility_stations_per_vtt(gdf_potential_mobility_stations_with_data, vtt, output_type=output_type) for vtt in range(0, 101)}
    gdf_potential_mobility_stations_with_data[EASTING] = gdf_potential_mobility_stations_with_data[GEOMETRY].x
    gdf_potential_mobility_stations_with_data[NORTHING] = gdf_potential_mobility_stations_with_data[GEOMETRY].y
    gdf_potential_mobility_stations_with_data = gdf_potential_mobility_stations_with_data.drop([KOSTEN_CHF], axis=1)
    return best_mobility_stations_per_vtt, gdf_potential_mobility_stations_with_data

### Execute

In [49]:
orig_lon_lat = (7.423570, 46.936620)
dest_lon_lat = (7.343680184933122, 46.551891386747386)

best_mobility_stations_per_vtt, gdf_potential_mobility_stations_with_data = get_best_mobility_statons_per_vtt(orig_lon_lat, dest_lon_lat)

### Visualize situation on map

In [ ]:
bounds = gdf_potential_mobility_stations_with_data.total_bounds
bounds = [[bounds[1], bounds[0]], [bounds[3], bounds[2]]]
bounds

In [ ]:
def show_best_mobility_stations_per_vtt(vtt):
    geo_data = GeoData(geo_dataframe=best_mobility_stations_per_vtt[vtt],
        style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
        hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
        point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
        name = 'Mobility-Stationen')
    for l in m.layers:
        if type(l) == GeoData:
            m.remove_layer(l)
    m.add_layer(geo_data)

In [ ]:
def on_slider_changed(event):
    show_best_mobility_stations_per_vtt(event["new"])

In [ ]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl, FullScreenControl, Marker, WidgetControl
from ipywidgets import IntSlider
import geopandas
import json

vtt_slider = IntSlider(description='Zeitkosten', min=0, max=100, value=20)
vtt_slider.observe(on_slider_changed, names='value')

widget_control = WidgetControl(widget=vtt_slider, position='topright')

m = Map(center=(52.3,8.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap, scroll_wheel_zoom=True)
m.add_control(widget_control)

m.fit_bounds(bounds)
show_best_mobility_stations_per_vtt(vtt_slider.value)
m.add_control(FullScreenControl())

m.layout.width = '100%'
m.layout.height = '500px'

orig_marker = Marker(location=(orig_lon_lat[1], orig_lon_lat[0]) , draggable=False)
dest_marker = Marker(location=(dest_lon_lat[1], dest_lon_lat[0]) , draggable=False)

m.add_layer(orig_marker)
m.add_layer(dest_marker)

m